In [2]:
import os
os.chdir( "../")

In [2]:
import psutil
num_cpus = psutil.cpu_count(logical=False)
num_cpus 


8

In [3]:
import pandas as pd
import numpy as np

path = "Dataset/data_train.csv"
df = pd.read_csv(filepath_or_buffer=path,
                               sep=",",
                               header=1,
                               engine='python',
                               names=['UserID', 'ItemID', 'Interaction'])


df

,UserID,ItemID,Interaction
0,1,15,1.0
1,1,16,1.0
2,1,133,1.0
3,1,161,1.0
4,1,187,1.0
...,...,...,...
478724,13024,13605,1.0
478725,13024,13823,1.0
478726,13024,15122,1.0
478727,13024,18185,1.0


In [4]:
user_ids = df["UserID"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
userencoded2user = {i: x for i, x in enumerate(user_ids)}
item_ids = df["ItemID"].unique().tolist()
item2item_encoded = {x: i for i, x in enumerate(item_ids)}
item_encoded2item = {i: x for i, x in enumerate(item_ids)}
df["User"] = df["UserID"].map(user2user_encoded)
df["Item"] = df["ItemID"].map(item2item_encoded)

num_users = len(user2user_encoded)
num_items = len(item_encoded2item)
df["Interaction"] = df["Interaction"].values.astype(np.float32)

# min and max ratings will be used to normalize the ratings later
min_rating = 0.0
max_rating = max(df["Interaction"])

print(
    "Number of users: {}, Number of Items: {}, Min rating: {}, Max rating: {}".format(
        num_users, num_items, min_rating, max_rating
    )
)

Number of users: 12638, Number of Items: 22222, Min rating: 0.0, Max rating: 1.0


In [5]:
userId_unique = df["UserID"].unique()
itemId_unique = df["ItemID"].unique()

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
import numpy as np
import scipy.sparse as sps
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

seeds = [
    1811854, 3772146, 6407100, 6926227, 1340378, 
    68679, 3822303, 4058970, 1264160, 1100338, 
    6552126, 725363, 8362134, 9446122, 5256199, 
    4686339, 3830906, 7315366, 9428914, 5564236, 
    3420043, 9223036, 851434, 3300082, 7899660, 
    2053768, 4305274, 3709256, 8795594, 1418903, 
    7428566, 157832
]

seeds = seeds[:num_cpus]

urm_all = sps.coo_matrix((df["Interaction"].values, 
                          (df["User"].values, df["Item"].values)))

urm_train_validation = []
urm_test = []
urm_train = []
urm_validation = []

for i in range(len(seeds)):
    a, b = split_train_in_two_percentage_global_sample(urm_all, train_percentage = 0.80,seed=seeds[i])
    c, d = split_train_in_two_percentage_global_sample(a, train_percentage = 0.80,seed=seeds[i])
    urm_train_validation.append(a)
    urm_test.append(b)
    urm_train.append(c)
    urm_validation.append(d)



In [7]:
num_users = len(userId_unique)
num_items = len(itemId_unique)

In [8]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = []
for u in urm_test:
    evaluator_test.append(EvaluatorHoldout(u, cutoff_list=[10], ignore_users=[]))

EvaluatorHoldout: Ignoring 2166 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2150 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2190 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2194 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2165 (17.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2145 (17.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2171 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2198 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [9]:
import os

output_folder_path = "result_experiments_xg/"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 1000  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [3]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

from HyperparameterTuning.run_hyperparameter_search import runHyperparameterSearch_Collaborative, runHyperparameterSearch_Content
from Recommenders.NonPersonalizedRecommender import TopPop, Random
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.Recommender_import_list import *
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
import os, multiprocessing
from functools import partial
from skopt.space import Real, Integer, Categorical


2024-01-08 10:09:48.498874: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-01-08 10:09:48.498899: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [11]:
from concurrent.futures import ProcessPoolExecutor
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
import time
from Recommenders.Incremental_Training_Early_Stopping import Incremental_Training_Early_Stopping
from Recommenders.MatrixFactorization.ImplicitIALSRecommender import ImplicitALSRecommender


In [12]:
import optuna

host = "192.168.16.83"
port = 3306
database_name = "optuna_mio"
username = "optuna"
password = "your_password"

# Create an SQLAlchemy engine
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_name}"
mysql_url
storage = optuna.storages.RDBStorage(url=mysql_url)

study_name = "RP3_8FOLD"

study = optuna.create_study(study_name=study_name, storage=storage, load_if_exists=True, direction="maximize", sampler=optuna.samplers.CmaEsSampler())

model = RP3betaRecommender

[I 2024-01-08 10:02:02,333] Using an existing study with name 'P3Alpha_8FOLD' instead of creating a new one.


In [13]:
def run_fold(fold, **hyperparams):
    recommender = model(urm_train_validation[fold])
    recommender.fit(**hyperparams)

    metric_result = evaluator_test[fold].evaluateRecommender(recommender)[0][metric_to_optimize].item()
    res = {'Fold': fold, metric_to_optimize: metric_result, **hyperparams}
    print(res)
    return res

def run_fold_wrapper(fold, hyperparams):
    return run_fold(fold, **hyperparams)

sum_map = lambda results: sum(item[metric_to_optimize] for item in results)


def objective(trial):

    params = {
        "topK": trial.suggest_int("topK",1, 1000),
        "alpha": trial.suggest_float("alpha",low = 0, high = 2),
        "beta": trial.suggest_float("beta",low = 0, high = 2),
    }

    with ProcessPoolExecutor(max_workers=num_cpus) as executor:
        # Submit all tasks to the process pool and wait for them to complete
        #fold_results = list(executor.map(lambda fold: run_fold(fold, **current_fit_hyperparameters_dict), range(len(seeds))))
        fold_results = list(executor.map(run_fold_wrapper, range(len(seeds)), [params]*len(seeds)))
    print(fold_results)
    average_score = sum_map(fold_results) / len(fold_results)
    return average_score

In [14]:
# param_id, trial_id, param_name, param_value, distribution_json, value
study.enqueue_trial({'alpha': 0.2765812198526 , 'beta': 0.1793692584175053, 'topK': 23})

study.optimize(objective, n_trials=200)

P3alphaRecommender: URM Detected 223 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.P3alphaRecommender: URM Detected 243 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 201 ( 1.6%) users with no interactions.P3alphaRecommender: URM Detected 239 ( 1.9%) users with no interactions.


P3alphaRecommender: URM Detected 122 ( 0.5%) items with no interactions.

P3alphaRecommender: URM Detected 229 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 220 ( 1.7%) users with no interactions.

P3alphaRecommender: URM Detected 109 ( 0.5%) items with no interactions.P3alphaRecommender: URM Detected 106 ( 0.5%) items with no interactions.


P3alphaRecommender: URM Detected 109 ( 0.5%) items with no interactions.P3alphaRecommender: URM Detected 114 ( 0.5%) items with no interactions.P3alphaRecommender: URM Detected 118 ( 0.5%) items wit

[I 2024-01-08 10:02:15,261] Trial 2 finished with value: 0.04207924823171881 and parameters: {'topK': 50, 'alpha': 0.7745100325869}. Best is trial 2 with value: 0.04207924823171881.


[{'Fold': 0, 'MAP': 0.042755458243904115, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 1, 'MAP': 0.04228147361156576, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 2, 'MAP': 0.04210906545492158, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 3, 'MAP': 0.042676105518168604, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 4, 'MAP': 0.04208064501461856, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 5, 'MAP': 0.04152745821477405, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 6, 'MAP': 0.04183271764168922, 'topK': 50, 'alpha': 0.7745100325869}, {'Fold': 7, 'MAP': 0.041371062154108615, 'topK': 50, 'alpha': 0.7745100325869}]
P3alphaRecommender: URM Detected 243 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 201 ( 1.6%) users with no interactions.P3alphaRecommender: URM Detected 223 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 239 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3al

[I 2024-01-08 10:02:41,619] Trial 3 finished with value: 0.022088496839890275 and parameters: {'topK': 420, 'alpha': 1.095128944847853}. Best is trial 2 with value: 0.04207924823171881.


[{'Fold': 0, 'MAP': 0.021974721706864524, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 1, 'MAP': 0.021985061536691948, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 2, 'MAP': 0.02219263503002018, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 3, 'MAP': 0.02229295471539819, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 4, 'MAP': 0.022967892494532398, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 5, 'MAP': 0.021575385101783653, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 6, 'MAP': 0.022427178539961578, 'topK': 420, 'alpha': 1.095128944847853}, {'Fold': 7, 'MAP': 0.02129214559386973, 'topK': 420, 'alpha': 1.095128944847853}]
P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 229 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 243 ( 1.9%) users with no interactions.
P3alphaRecommender: URM Detected 239 ( 1.9%) use

[I 2024-01-08 10:02:55,916] Trial 4 finished with value: 0.03682648397259192 and parameters: {'topK': 198, 'alpha': 0.7183189363041375}. Best is trial 2 with value: 0.04207924823171881.


[{'Fold': 0, 'MAP': 0.03665111499145151, 'topK': 198, 'alpha': 0.7183189363041375}, {'Fold': 1, 'MAP': 0.03659711628104123, 'topK': 198, 'alpha': 0.7183189363041375}, {'Fold': 2, 'MAP': 0.03680471237754973, 'topK': 198, 'alpha': 0.7183189363041375}, {'Fold': 3, 'MAP': 0.03676283337284921, 'topK': 198, 'alpha': 0.7183189363041375}, {'Fold': 4, 'MAP': 0.03733530211473549, 'topK': 198, 'alpha': 0.7183189363041375}, {'Fold': 5, 'MAP': 0.03689637763043877, 'topK': 198, 'alpha': 0.7183189363041375}, {'Fold': 6, 'MAP': 0.03739444906971452, 'topK': 198, 'alpha': 0.7183189363041375}, {'Fold': 7, 'MAP': 0.03616996594295485, 'topK': 198, 'alpha': 0.7183189363041375}]
P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.P3alphaRecommender: URM Detected 239 ( 1.9%) users with no interactions.P3alphaRecommender: URM Detected 223 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 229 ( 1.8%) us

[I 2024-01-08 10:03:12,475] Trial 5 finished with value: 0.024490818878622214 and parameters: {'topK': 149, 'alpha': 1.003134659945212}. Best is trial 2 with value: 0.04207924823171881.


[{'Fold': 0, 'MAP': 0.024342324050832424, 'topK': 149, 'alpha': 1.003134659945212}, {'Fold': 1, 'MAP': 0.024049525988885212, 'topK': 149, 'alpha': 1.003134659945212}, {'Fold': 2, 'MAP': 0.024430383881474987, 'topK': 149, 'alpha': 1.003134659945212}, {'Fold': 3, 'MAP': 0.024753253177946754, 'topK': 149, 'alpha': 1.003134659945212}, {'Fold': 4, 'MAP': 0.02531924495187169, 'topK': 149, 'alpha': 1.003134659945212}, {'Fold': 5, 'MAP': 0.024339941669351755, 'topK': 149, 'alpha': 1.003134659945212}, {'Fold': 6, 'MAP': 0.024777164360855978, 'topK': 149, 'alpha': 1.003134659945212}, {'Fold': 7, 'MAP': 0.023914712947758942, 'topK': 149, 'alpha': 1.003134659945212}]
P3alphaRecommender: URM Detected 227 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 229 ( 1.8%) users with no interactions.P3alphaRecommender: URM Detected 223 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 217 ( 1.7%) users with no interactions.P3alphaRecommender: URM Detected 239 ( 1.9%) us